# BEFORE YOU START

## Step 1: Try starting a GPU-enabled session. (`Runtime > Change runtime type > Dropdown "GPU"`)
## Step 2: Run the cell below.

In [ ]:
%pip show torch

## Step 3: Replace "_version" variable with the correct version, printed using the above command. (Explicit versioning is a workaround; google's runtime environment tends to cause `pip install torch-sparse` to hang for an hour.)

In [ ]:
# Replace this with the correct torch version, shown above.
_version = "1.13.1+cu116"

## Step 4: Fill in your email address, e.g. `_email = "test_email@myaddress.edu"` (to be used for Entrez API).

Note: We do not store or track anything using your e-mail address. It is to be specified for <a href="https://www.ncbi.nlm.nih.gov/books/NBK25501/">NCBI's Entrez API</a>; see <a href="https://www.ncbi.nlm.nih.gov/books/NBK25497/#chapter2.Usage_Guidelines_and_Requiremen">Usage guidelines and requirements</a>.

In [ ]:
# Specify your e-mail for NCBI's API.
_email = ""

# Install the package.

In [ ]:
# ========================== Do not modify below. =======================
# Colab-specific installation commands
_pyg_url = "https://pytorch-geometric.com/whl/torch-{}.html".format(_version)
%pip install torch-sparse torch-scatter -f $_pyg_url
!sudo apt -y install bwa

# ========================== Install chronostrain. =======================
!git clone https://github.com/gibsonlab/chronostrain
%pip install -e chronostrain/.

# Set up the test environment.

In [ ]:
%cp -r chronostrain/examples/colab_demo/configs ./configs
!unzip chronostrain/examples/colab_demo/inputs.zip -d ./

# Example usage

Display the help text for chronostrain and exit.

In [ ]:
!chronostrain -h

# Running chronostrain:

Use the CHRONOSTRAIN_INI environment variable (or the -c option) to pass in a configuration file, along with a command.

In [ ]:
%env CHRONOSTRAIN_INI=configs/chronostrain.ini
%env ENTREZ_EMAIL=$_email
%env CHRONOSTRAIN_LOG_INI=chronostrain/examples/example_configs/log_config.ini

!chronostrain filter -r ./inputs/test_reads.csv -o ./output/filtered --aligner bwa

In [ ]:
!chronostrain advi -r output/filtered/filtered_test_reads.csv -o output/inference